In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/kaggle/input/anomaly-detection-in-helicopter/train_df.csv")

In [3]:
df.isnull().sum().sum()

0

In [4]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,61430,61431,61432,61433,61434,61435,61436,61437,61438,61439
0,0,0.041259,0.041259,0.032573,0.023887,0.029315,0.041259,0.045602,0.038001,0.030401,...,0.034744,-0.007600,-0.003257,0.065145,0.047773,-0.009772,0.031487,0.096632,0.077089,0.040173
1,1,-0.211722,-0.264924,-0.274696,-0.236694,-0.156349,-0.059716,0.005429,0.046687,0.153091,...,0.260581,-0.004343,-0.241037,-0.636252,-0.953292,-0.980436,-0.846888,-0.838202,-0.880546,-0.739398
2,2,0.214105,0.154930,0.136640,0.013987,-0.038733,-0.015063,-0.111894,-0.104363,0.047340,...,0.040884,0.375490,0.699337,0.965085,1.086662,1.132926,1.279249,1.296464,0.937112,0.451879
3,3,-0.154837,-0.127768,-0.217638,-0.284770,-0.299929,-0.270694,-0.077960,0.092036,0.076877,...,-0.286936,-0.171079,-0.036814,-0.024904,0.031400,0.140761,-0.011911,-0.173244,-0.063884,0.081208
4,4,-1.022780,-0.916376,-0.676425,-0.461445,-0.330069,-0.122690,0.178064,0.489675,0.799115,...,1.009751,1.134613,0.836030,0.479903,0.109661,-0.285553,-0.628651,-0.916376,-1.010837,-0.804544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,1672,-0.063989,0.017228,0.015997,-0.321177,-0.503300,-0.535295,-0.562367,-0.600515,-0.573442,...,-0.121826,-0.134131,-0.111981,-0.057836,-0.073834,-0.125517,-0.082448,-0.036917,-0.114442,-0.183354
1673,1673,0.993037,0.726052,0.341885,0.286314,0.535177,0.605245,0.198124,-0.375711,-0.579875,...,0.038658,0.188460,0.273025,0.045907,-0.108727,0.072484,0.415577,0.356382,-0.096646,-0.280273
1674,1674,0.570550,0.253578,-0.193841,-0.502279,-0.657108,-0.838757,-0.971642,-0.881427,-0.700996,...,-0.146295,0.128008,0.162143,0.018287,-0.017068,0.087777,0.062175,-0.041450,-0.140199,-0.247482
1675,1675,0.630677,0.605010,0.477897,0.155225,-0.441229,-0.679566,-0.515786,-0.424118,-0.183336,...,0.229781,-0.041556,-0.176003,-0.301894,-0.177225,0.033001,0.025667,0.088001,0.206559,0.003667


# # Deep learning AutoEncoder

In [6]:
from keras.layers import Dense,LSTM,SimpleRNN,GRU,BatchNormalization,TimeDistributed,Input,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [7]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [15]:
df_reshaped=df.values.reshape(df.shape[0],1,df.shape[1])
df_reshaped.shape

(1677, 1, 61441)

In [19]:
inputs=Input(shape=(1,61441))
E=SimpleRNN(600,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
E=Dropout(0.5)(E)
E=SimpleRNN(500,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=Dropout(0.5)(E)
E=LSTM(300,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
E=Dropout(0.5)(E)
E=LSTM(200,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(E)
D=Dropout(0.5)(E)

D=SimpleRNN(200,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(E)
D=SimpleRNN(300,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(E)
D=LSTM(400,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dropout(0.5)(E)
D=LSTM(500,activation="relu",return_sequences=True,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=TimeDistributed(Dense(61441,activation="sigmoid"))(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["mae"])

In [21]:
model.fit(df_reshaped,df_reshaped,epochs=50,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - loss: 31.4700 - mae: 0.5472 - val_loss: 19.8849 - val_mae: 0.4368
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - loss: 16.9500 - mae: 0.5595 - val_loss: 9.4295 - val_mae: 0.4310
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - loss: 8.1630 - mae: 0.5445 - val_loss: 5.1785 - val_mae: 0.4316
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - loss: 4.6755 - mae: 0.5824 - val_loss: 3.2889 - val_mae: 0.4308
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - loss: 3.0231 - mae: 0.5467 - val_loss: 2.2377 - val_mae: 0.4317
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - loss: 2.1445 - mae: 0.5545 - val_loss: 1.6477 - val_mae: 0.4314
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - loss: 1.6486 - mae: 0.5733 - val_loss: 1.2215 - val_mae: 0.4310
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - loss: 1.1745 - mae: 0.5592 - val_loss: 0.9431 - val_mae: 0.4312
Epoch 9/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - loss: 1.0266 - ma

In [23]:
from sklearn.metrics import mean_squared_error
import numpy as np
reconstructed_data = model.predict(df_reshaped)
reconstruction_error = np.mean(np.square(df_reshaped
                                         - reconstructed_data), axis=1)

threshold = np.percentile(reconstruction_error, 95)  # e.g., 95th percentile

labels = (reconstruction_error > threshold).astype(int)  # 1 for anomaly, 0 for normal
print("Anomaly labels:", labels)


53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step
Anomaly labels: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [24]:
np.argmax(labels)

126370

In [36]:
np.sum(labels)

5151828

In [26]:
np.unique(labels)

array([0, 1])

In [27]:
d=pd.DataFrame(labels)

In [28]:
d

,0,1,2,3,4,5,6,7,8,9,...,61431,61432,61433,61434,61435,61436,61437,61438,61439,61440
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1673,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1674,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1675,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
np.argmax(labels)

126370